In [2]:
from stable_baselines import DQN
from stable_baselines.gail import generate_expert_traj

model = DQN('MlpPolicy', 'CartPole-v1', verbose=1)
      # Train a DQN agent for 1e5 timesteps and generate 10 trajectories
      # data will be saved in a numpy archive named `expert_cartpole.npz`
generate_expert_traj(model, 'generated_traj_data/expert_cartpole', n_timesteps=int(1e5), n_episodes=10)

Creating environment from the given name, wrapped in a DummyVecEnv.






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








--------------------------------------
| % time spent exploring  | 75       |
| episodes                | 100      |
| mean 100 episode reward | 24.9     |
| steps                   | 2463     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 200      |
| mean 100 episode reward | 81.5     |
| steps                   | 10617    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 300      |
| mean 100 episode reward | 118      |
| steps                   | 22381    |
----------------------------

{'actions': array([[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1]]),
 'obs': array([[-0.04735008, -0.01295144, -0.03380163, -0.00572591],
        [-0.0476091 , -0.20757273, -0.03391615,  0.27610347],
        [-0.05176056, -0.4021948 , -0.02839408,  0.55789924],
        ...,
        [ 0.471994  ,  0.17146152, -0.00464677, -0.24576311],
        [ 0.47542325, -0.02359376, -0.00956203,  0.0454505 ],
        [ 0.47495136, -0.2185773 , -0.00865302,  0.33510125]],
       dtype=float32),
 'rewards': array([1., 1., 1., ..., 1., 1., 1.]),
 'episode_returns': array([500., 500., 500., 500., 500., 500., 500., 500., 500., 500.]),
 'episode_starts': array([ True, False, False, ..., False, False, False])}

In [ ]:
model.learn()

In [2]:
# pretraining using the saved trajectory

In [5]:
from stable_baselines import PPO2
from stable_baselines.gail import ExpertDataset
# Using only one expert trajectory
# you can specify `traj_limitation=-1` for using the whole dataset
dataset = ExpertDataset(expert_path='generated_traj_data/expert_cartpole.npz',
                        traj_limitation=-1, batch_size=128)


actions (5000, 1)
obs (5000, 4)
rewards (5000,)
episode_returns (10,)
episode_starts (5000,)
Total trajectories: -1
Total transitions: 5000
Average returns: 500.0
Std for returns: 0.0


In [25]:
dataset.init_dataloader(128)

In [30]:
# dataset.get_next_batch()[0]

In [31]:
# (s, a) -> (s+1, r)


# TODO check assume not expert in IL
(s0, s500) -> (a0, a500) 

In [14]:
dataset.__dict__.keys()

dict_keys(['observations', 'actions', 'returns', 'avg_ret', 'std_ret', 'verbose', 'num_traj', 'num_transition', 'randomize', 'sequential_preprocessing', 'dataloader', 'train_loader', 'val_loader'])

In [32]:
model = PPO2('MlpPolicy', 'CartPole-v1', verbose=1)
# Pretrain the PPO2 model
model.pretrain(dataset, n_epochs=1000)


Creating environment from the given name, wrapped in a DummyVecEnv.



Pretraining with Behavior Cloning...
==== Training progress 10.00% ====
Epoch 100
Training loss: 0.346077, Validation loss: 0.308957

==== Training progress 20.00% ====
Epoch 200
Training loss: 0.320585, Validation loss: 0.280822

==== Training progress 30.00% ====
Epoch 300
Training loss: 0.321907, Validation loss: 0.279914

==== Training progress 40.00% ====
Epoch 400
Training loss: 0.323388, Validation loss: 0.279585

==== Training progress 50.00% ====
Epoch 500
Training loss: 0.322113, Validation loss: 0.279030

==== Training progress 60.00% ====
Epoch 600
Training loss: 0.319191, Validation loss: 0.279283

==== Training progress 70.00% ====
Epoch 700
Training loss: 0.317897, Validation loss: 0.278338

==== Training progress 80.00% ====
Epoch 800
Training loss: 0.318741, Validation loss: 0.280931

==== Training progress 90.00% ====
Epoch 900
Training loss: 0.319314, Validation loss: 0.280736

==== Training progr

In [34]:


# As an option, you can train the RL agent
# model.learn(int(1e5))

# Test the pre-trained model
env = model.get_env()
obs = env.reset()

reward_sum = 0.0
for _ in range(1000):
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward
        env.render()
        if done:
                print(reward_sum)
                reward_sum = 0.0
                obs = env.reset()

env.close()

[500.]
[500.]
